# General plots for the manuscript and visualization of data statistics
## Warning: This notebook takes approximately 20 minutes to run because all data needs to be loaded and processed.

In [ ]:
import os

import gpytorch
import matplotlib.pyplot as plt
import numpy as np
import torch

from src import config as cfg
from src.batt_data import batt_data, data_utils
from src.gp.wiener_kernel import WienerKernel
from src.plot_utils import ocv_lookup_plot
from src.exceptions import InsufficientDataError
from src.path_setup import setup_paths


%load_ext autoreload
%autoreload 2

# Use seaborn style defaults and set the default figure size
plt.style.use("seaborn-v0_8-white")

setup_paths()
data_utils.build_data_cache()

In [ ]:
# Define your OCV-SOC curves here to avoid loading the files with the data multiple times
# Simple SOC curve from publication xyz.
cell_characteristics = data_utils.read_cell_characteristics()
ocv_lookup_plot(cell_characteristics)

# Random Draws from GP Kernel

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model_se = ExactGPModel(
    torch.tensor([0]),
    torch.tensor([0]),
    likelihood,
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()),
)

model_wiener = ExactGPModel(
    torch.tensor([0.0]),
    torch.tensor([0.0]),
    likelihood,
    gpytorch.kernels.ScaleKernel(WienerKernel()),
)

test_x_se = torch.linspace(20, 25, 100)
test_x_wiener = torch.linspace(0, 5, 100)

model_se.eval()
model_wiener.eval()

with torch.no_grad():
    out_se = model_se(test_x_se)

with torch.no_grad():
    out_wiener = model_wiener(test_x_wiener)

f_samples_se = out_se.sample(sample_shape=torch.Size((20,))).detach().numpy()
f_samples_wiener = out_wiener.sample(sample_shape=torch.Size((20,))).detach().numpy()

In [ ]:
# PLot fsamples
def plot_fsamples(f_samples, test_x, kernel_name):
    plt.figure(figsize=(10, 6))
    for sample in f_samples:
        plt.plot(test_x.numpy(), sample, lw=2, color="k", alpha=np.random.rand())

    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)
    # Turn axes off
    ax.axes.get_yaxis().set_visible(False)
    ax.axes.get_xaxis().set_visible(False)
    # remove
    plt.savefig(f"results/data_vis/{kernel_name}_samples.pdf", bbox_inches="tight")


plot_fsamples(f_samples_se, test_x_se, "se")
plot_fsamples(f_samples_wiener, test_x_wiener, "wiener")

# Data Overview and Statistics
In the following code section, statistical histograms for the data set is generated

In [5]:
# Write a function to integrate the current throughput over time to get the total charge throughput
def integrate_current_throughput(df, Ibat_col):
    # Integrate the current throughput over time to get the total charge throughput
    Q_throughput = 0
    delta_t = 0
    last_time = df.index[0]
    # Create a mask where the current is negative
    mask = df[Ibat_col] < 0
    for ind, row in df[mask].iterrows():
        delta_t = ind - last_time
        delta_t = delta_t.total_seconds()
        if delta_t < 10:
            Q_throughput += (-1) * (row[Ibat_col]) * delta_t
        last_time = ind

    Q_throughput = Q_throughput / 3600
    return Q_throughput

# Historgram statistics of the data set
## For the ridgelineplot please run `data_vis_ridgeline_plot.py`
## The histogram cell takes about 15 minutes to run.

In [ ]:
columns_eval = ["T_Cell_1_2", "T_Cell_3_4", "T_Cell_5_6", "T_Cell_7_8", "I_Batt", "U_Batt", "SOC_Batt"]
cell_volt_container = []
ibat_container = []
soc_container = []
temp_container = []
bat_age_container = []
sampling_time_container = []
batt_id_container = []
current_throughput_container = []

for batt_id in range(1, 29):
    batt_id_container.append(batt_id)
    volt_cols = (
        list(["U_Cell_" + str(cellnr) for cellnr in range(1, 9)])
    )
    df = data_utils.read_battery_data(
        batt_id, keep_columns=columns_eval + volt_cols
    )
    # get all voltage data
    cell_volt_container.append(df[volt_cols].to_numpy().flatten())
    # get all ibat data
    ibat_container.append(df["I_Batt"].to_numpy().flatten())
    # get all soc data
    soc_container.append(df["SOC_Batt"].to_numpy().flatten())
    # get all temperature data
    temp_container.append(
        df[["T_Cell_1_2", "T_Cell_3_4", "T_Cell_5_6", "T_Cell_7_8"]].to_numpy().flatten()
    )
    # get all battery age data
    bat_age_container.append((df.index[-1] - df.index[0]).days)
    # get all sampling time data
    sampling_times_series = df.index.to_series().diff().dt.total_seconds()
    sampling_time_container.append(sampling_times_series.to_numpy().flatten())

    current_throughput_container.append(integrate_current_throughput(df, "I_Batt"))
# merge all voltage data
cells_volt_data = np.concatenate(cell_volt_container)
# clean NaNs
cells_volt_data = cells_volt_data[~np.isnan(cells_volt_data)]
# merge all ibat data
ibat_data = np.concatenate(ibat_container)
# clean NaNs
ibat_data = ibat_data[~np.isnan(ibat_data)]
# merge all soc data
soc_data = np.concatenate(soc_container)
# clean NaNs
soc_data = soc_data[~np.isnan(soc_data)]
# merge all temperature data
temp_data = np.concatenate(temp_container)
# clean NaNs
temp_data = temp_data[~np.isnan(temp_data)]
# merge all battery age data
bat_age_data = np.asarray(bat_age_container)
# clean NaNs
bat_age_data = bat_age_data[~np.isnan(bat_age_data)]
# merge all sampling time data
sampling_time_data = np.concatenate(sampling_time_container)
# clean NaNs
sampling_time_data = sampling_time_data[~np.isnan(sampling_time_data)]
# plot histogram
# print max and min values
print(
    "voltage max: {}, min: {}".format(np.max(cells_volt_data), np.min(cells_volt_data))
)
print("ibat max: {}, min: {}".format(np.max(ibat_data), np.min(ibat_data)))
print("soc max: {}, min: {}".format(np.max(soc_data), np.min(soc_data)))
print("temp max: {}, min: {}".format(np.max(temp_data), np.min(temp_data)))
print("bat age max: {}, min: {}".format(np.max(bat_age_data), np.min(bat_age_data)))
print(
    "sampling time max: {}, min: {}".format(
        np.max(sampling_time_data), np.min(sampling_time_data)
    )
)
print("total number of data points: ", len(ibat_data))
# hist subplots for voltage, ibat, soc, temp
fig, axs = plt.subplots(2, 3, figsize=(14, 10))
fig.subplots_adjust(hspace=0.3, wspace=0.3)
# voltage
nb_bins = 100
axs[0, 0].hist(cells_volt_data, bins=nb_bins, color="blue", range=(0, 6))
axs[0, 0].set_xticks(np.arange(0, 7, 1))
axs[0, 0].set_title("$V_{cells}$")
axs[0, 0].set_xlabel("Voltage (V)")
axs[0, 0].set_yscale("log")
axs[0, 0].set_ylabel("Log count")
# increase tick size 
axs[0, 0].tick_params(axis="both", which="major")
# ibat
axs[0, 1].hist(ibat_data, bins=nb_bins, color="darkorange", range=(-750, 250))
axs[0, 1].set_title("Current")
axs[0, 1].set_xlabel("Current (A)")
axs[0, 1].set_yscale("log")
axs[0, 1].set_ylabel("Log count")
# battery age
axs[0, 2].hist(bat_age_data, bins=nb_bins, color="magenta")
axs[0, 2].set_title("Battery data recorded time")
axs[0, 2].set_xlabel("Days")
axs[0, 2].set_ylabel("Count")
# soc
axs[1, 0].hist(soc_data, bins=nb_bins, color="green", range=(0, 100))
axs[1, 0].set_title("SOC")
axs[1, 0].set_xlabel("SOC (%)")
axs[1, 0].set_yscale("log")
axs[1, 0].set_ylabel("Log count")
# temperature
axs[1, 1].hist(temp_data, bins=nb_bins, color="red", range=(-25, 60))
axs[1, 1].set_xticks(np.arange(-25, 100, 25))
axs[1, 1].set_title("Temperature")
axs[1, 1].set_xlabel("Temperature (°C)")
axs[1, 1].set_ylabel("Count")
# sampling time
axs[1, 2].hist(sampling_time_data, bins=nb_bins, color="black", range=(0, 60))
axs[1, 2].set_title("Sampling time")
axs[1, 2].set_xlabel("Sampling time (s)")
axs[1, 2].set_yscale("log")
axs[1, 2].set_ylabel("Log count")
plt.show()

os.makedirs(cfg.PATH_FIGURES_DATA_VIS, exist_ok=True)
fig.savefig(os.path.join(cfg.PATH_FIGURES_DATA_VIS, "histograms.pdf"), bbox_inches="tight")

# Create Table 2 for the manuscript

In [7]:
# Reload all data nd sleect subset of data according to the GP rules.
# Takes about 2 min to run
batt_age_container_post_selection = []
for batt_id in batt_id_container:
    try:
        battdata = batt_data.BattData(
            batt_id,
            cell_characteristics,
            segment_selection=True,
            gap_removal=cfg.GAP_REMOVAL,
        )
        batt_age_container_post_selection.append(battdata.age)
    except InsufficientDataError:
        batt_age_container_post_selection.append(0)

In [8]:
string = """
    (7) & Cell 7 behaves differently to the other cells.  \\ \hline
    (6), (8) & Cells 6 and 8 show a slightly higher resistance than the other cells. \\ \hline
    (7) & Cell 7 shows an upward resistance trend, while other cells have fairly constant resistances. \\ \hline
    (1) & Cell 1 shows an upward resistance trend, while other cells have fairly constant resistances.  \\ \hline
        & Flat resistances. Cell 1 shows a slightly higher resistance than the other cells when the mean of the selected data is used as a reference point (see Fig.\,S9).  \\ \hline
    (8) & Cell 8 has a slightly higher resistance than the other cells and shows a resistance knee after 3 years. \\ \hline
    (5), (7), (8)       & Cells 5, 7 and 8 show different trends in comparison to the other cells.   \\ \hline
        & All cells show a resistance increase close to the end of the available data. Seasonal temperature variations are affecting resistance estimates.  \\ \hline
    (2)     & After approx. 230 days: Cell 2 shows a knee-shaped resistance increase. \\ \hline
    (7), (8) & Cell 8 shows an increasing resistance after 100 days. Cell 7 behaves slightly differently to the other cells. \\ \hline
        & Lower average current than most other systems, unclear interpretation. \\ \hline
    (8), (7)       & Cells 8 and 7 show different behavior (see also Fig.\,S9). Significantly higher temperatures were measured by temperature sensor 4 (see Fig.\,S21). \\ \hline
        & Lower average current than most other systems, unclear interpretation.  \\ \hline
    (1)    & Cell 1 shows a lower resistance to the other cells.  \\ \hline
        & Many data gaps, unclear interpretation.  \\ \hline
    (4)       & Cell 4 shows a lower resistance to the other cells.   \\ \hline
    (3)       & Cell 3 shows a much higher resistance to the other cells. \\ \hline
        &  All cells show increasing resistance toward the end of use, unclear interpretation. \\ \hline
        &  Lower average current than most other systems, unclear interpretation.\\ \hline
        &  Lower average current than most other systems, unclear interpretation.\\ \hline
        &  All cells show increasing resistance toward the end of use, unclear interpretation. Temperature variations affect resistance estimates. \\ \hline
        & \\ \hline
        & \\ \hline
        & \\ \hline
        & \\ \hline
        & \\ \hline
        & \\ \hline
    """
strings = string.split("\hline")
# delete the "\\"
strings = [s.replace(" \\ ", "") for s in strings]
# delete the empty strings
strings = [s for s in strings if s]
# delete "\n"
strings = [s.replace("\n", "") for s in strings]
strings = [s.replace("     ", "") for s in strings]
#strings = [s.replace("   ", "") for s in strings]
#strings = [s.replace("  ", "") for s in strings]

In [ ]:
for battid, current, age, age_gp, string in zip(batt_id_container, current_throughput_container, bat_age_container, batt_age_container_post_selection, strings):
    print(f"{battid} & {current/160:.0f} & {age} & {age_gp:.0f} &" + string + r'\\ ' +  "\hline")

# instead write to a file
with open("results/data_vis/stats_table.txt", "w") as file:
    for battid, current, age, age_gp, string in zip(batt_id_container, current_throughput_container, bat_age_container, batt_age_container_post_selection, strings):
        file.write(f"{battid} & {current/160:.0f} & {age} & {age_gp:.0f} &" + string + r'\\ ' +  "\hline" + "\n")